In [1]:
from pyplasm import *
from larlib import *
import csv

Evaluating fenvs.py..
...fenvs.py imported in 0.0157015852821 seconds


c:\python27\lib\site-packages\larlib\larstruct.py:233: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  self.body = [item for item in data if item != None]


In [2]:
%run Workshop_01.ipynb

In [2]:
def intersperse(sequence, value):
    res = [value] * (2 * len(sequence) - 1)
    res[::2] = sequence
    return res


In [3]:
def getCenter(base, x):
    if (base > x):
        center = (base - x) / 2.0
    else:
        center = 0
        
    return center

def generateBeams(structParams, beamX):
    (by, bz), (px, py), distances, heights = structParams

    # get centering value
    centerY = getCenter(py, by)

    negate = lambda x: -x
    adjust = lambda x: negate(x + (2 * centerY))

    beams_y = [-centerY, by] + intersperse(map(adjust, distances), by) + [by]
    beams_z = intersperse(map(negate, heights), bz) + [bz]

    # generate beams
    res = PROD(
        [PROD([QUOTE([beamX + px]), QUOTE(beams_y)]),
         QUOTE(beams_z)])

    return res




In [ ]:
def ggpl_bone_structure(file_name):
    state = 0
    res = []
    params = None

    # should use a less dangerous parsing method than eval
    parseVect = lambda line: eval(line)
    parseList = lambda line: map(parseVect, line)

    with open(file_name) as f:
        for row in csv.reader(f):          
            # FSM used for parsing csv. 0: odd lines, 1: even lines    
            if state == 0:
                position = parseVect(*row)
                beamX = position[0]

                if params:
                    beams = generateBeams(params, beamX)
                    res.append(beams)

                res.append(T([1,2,3])(position))
                state = 1
                
            else:
                params = parseList(row)
                res.append(buildStructure(*params))
                state = 0

    return STRUCT(res)
VIEW(ggpl_bone_structure('data_frame_461395.csv'))